In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import os
import pandas as pd

In [2]:
news_url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [3]:
response1 = requests.get(news_url)
soup1 = bs(response1.text, 'lxml')

In [4]:
results1 = soup1.body.find("div", class_="slide")

In [5]:
news_title=results1.find('div',class_='content_title').text
news_p=results1.find('div',class_="rollover_description_inner").text
print(news_title,news_p)



NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network

 
Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.



In [6]:
image_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
base_image_url='https://www.jpl.nasa.gov'

In [7]:
response2 = requests.get(image_url)
soup2 = bs(response2.text, 'lxml')

In [8]:
featured_image= soup2.body.find("article", class_="carousel_item")['style']
print(featured_image)

background-image: url('/spaceimages/images/wallpaper/PIA19347-1920x1200.jpg');


In [9]:
featured_image=featured_image.replace("background-image: url('","")
featured_image=featured_image.replace("');","")
featured_image_url=base_image_url+featured_image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19347-1920x1200.jpg


In [10]:
weather_url='https://twitter.com/marswxreport?lang=en'

In [11]:
response3 = requests.get(weather_url)
soup3 = bs(response3.text, 'lxml')

In [12]:
tweet=soup3.body.find("li", {"data-item-type": "tweet"})
tweet_text_box = tweet.find("p", {"class": "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"})
images_in_tweet_tag = tweet_text_box.find_all("a", {"class": "twitter-timeline-link u-hidden"})
tweet_text = tweet_text_box.text
for image_in_tweet_tag in images_in_tweet_tag:
    mars_weather = tweet_text.replace(image_in_tweet_tag.text, '')
print(mars_weather)

InSight sol 445 (2020-02-26) low -92.8ºC (-135.0ºF) high -12.8ºC (8.9ºF)
winds from the SSE at 5.9 m/s (13.3 mph) gusting to 21.1 m/s (47.3 mph)
pressure at 6.30 hPa


In [13]:
fact_url='https://space-facts.com/mars/'

In [14]:
tables=pd.read_html(fact_url)
df=tables[0]

In [15]:
df.rename(columns={ df.columns[0]: "Description" }, inplace = True)
df.rename(columns={ df.columns[1]: "Value" }, inplace = True)
df.set_index('Description',inplace=True)
df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
html_table=df.to_html
df.to_html('table.html')

In [18]:
soup_table = bs(open("table.html"), "html.parser")
soup_table

<table border="1" class="dataframe">
<thead>
<tr style="text-align: right;">
<th></th>
<th>Value</th>
</tr>
<tr>
<th>Description</th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<th>Equatorial Diameter:</th>
<td>6,792 km</td>
</tr>
<tr>
<th>Polar Diameter:</th>
<td>6,752 km</td>
</tr>
<tr>
<th>Mass:</th>
<td>6.39 × 10^23 kg (0.11 Earths)</td>
</tr>
<tr>
<th>Moons:</th>
<td>2 (Phobos &amp; Deimos)</td>
</tr>
<tr>
<th>Orbit Distance:</th>
<td>227,943,824 km (1.38 AU)</td>
</tr>
<tr>
<th>Orbit Period:</th>
<td>687 days (1.9 years)</td>
</tr>
<tr>
<th>Surface Temperature:</th>
<td>-87 to -5 °C</td>
</tr>
<tr>
<th>First Record:</th>
<td>2nd millennium BC</td>
</tr>
<tr>
<th>Recorded By:</th>
<td>Egyptian astronomers</td>
</tr>
</tbody>
</table>

In [46]:
th=[]
td=[]
for x in range(9):
    th.append(soup_table.tbody.find_all('th')[x].text)
    td.append(soup_table.tbody.find_all('td')[x].text)
table_title_1=soup_table.find_all('th')[1].text
table_title_2=soup_table.find_all('th')[2].text

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [14]:
hemisphere_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [35]:
browser.visit(hemisphere_url)

In [38]:
hemisphere_image_urls=[]
html=browser.html
for x in range(1,5):
    browser.click_link_by_partial_text('Enhanced')
    soup=bs(browser.html,'html.parser')
    #print('New HTML', browser.html)
    title=soup.find('h2',class_='title').text.replace(' Enhanced','')
    # print('New Title', type(title))
    img_url=soup.find('div',class_='downloads').find('a')['href']
    list={
        'title':title,
        'img_url':img_url
    }
    hemisphere_image_urls.append(list)

[{'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
